In [1]:
from datasets import load_dataset
import pandas as pd
import torch
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
# pose sequence as a NLI premise and label as a hypothesis
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AdamW
from datasets import load_metric
import torch.nn.functional as F

giga_dataset = load_dataset("gigaword", split='train[:1%]')

Using custom data configuration default
Reusing dataset gigaword (/tmp/xdg-cache/huggingface/datasets/gigaword/default/1.2.0/c518c578e42a6afe842b09e979ee2907ea42a12b57ba992fae9e9d7347825245)


In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

nli_model = AutoModelForSequenceClassification.from_pretrained('facebook/bart-large-mnli').to(device)
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-mnli')

Some weights of the model checkpoint at facebook/bart-large-mnli were not used when initializing BartForSequenceClassification: ['model.encoder.version', 'model.decoder.version']
- This IS expected if you are initializing BartForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BartForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
giga_filtered_keys = pd.DataFrame()

def mapper(examples):
    return tokenizer([(examples['document'], examples['summary'])], return_tensors='pt', truncation=True, padding='max_length')
print(giga_dataset)

gigaset = giga_dataset.map(mapper)

gigaset.set_format(type='torch', columns=['input_ids', 'attention_mask'])
print(gigaset[0])

Dataset({
    features: ['document', 'summary'],
    num_rows: 38040
})



{'attention_mask': [tensor([1, 1, 1,  ..., 0, 0, 0])], 'input_ids': [tensor([   0,  102, 4193,  ...,    1,    1,    1])]}


In [4]:
giga_filtered_keys = pd.DataFrame()

giga_loader = DataLoader(gigaset, batch_size=4)

print(next(iter(giga_loader)))

with torch.no_grad():
    all_preds = []
    for batch in giga_loader:
        batch = {k: v[0].to(device) for k, v in batch.items()}
        outputs = nli_model(**batch)
        preds = torch.argmax(F.softmax(outputs[0][:,[0,2]], dim=1), dim=1)
        all_preds.append(preds)
        
    giga_filtered_keys['index_keys'] = torch.cat(all_preds, dim=0).cpu()
    
print(giga_filtered_keys)

{'attention_mask': [tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])], 'input_ids': [tensor([[    0,   102,  4193,  ...,     1,     1,     1],
        [    0,   415,   513,  ...,     1,     1,     1],
        [    0,   102,  4193,  ...,     1,     1,     1],
        [    0, 25515,   449,  ...,     1,     1,     1]])]}
       index_keys
0               1
1               1
2               1
3               1
4               1
...           ...
38035           1
38036           1
38037           1
38038           1
38039           1

[38040 rows x 1 columns]


In [5]:
# for i,item in enumerate(giga_filtered_keys['index_keys']):
#     if item == 0:
#         print(i)

In [6]:
giga_filtered_keys.to_csv('./gigawordfiltered.csv')